- install langchain, langchain-community, langchain-google-genai, dotenv

In [51]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.messages import HumanMessage, SystemMessage, AIMessage

In [ ]:
# load environment variables from .env file
load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")

In [5]:
# inisiasi model
llm = ChatGoogleGenerativeAI(google_api_key=google_api_key, model="gemini-2.0-flash")

In [7]:
llm.model

'models/gemini-2.0-flash'

In [8]:
llm.invoke('Halo nama saya Rizki dan saya ingin belajar tentang LangChain.')

AIMessage(content='Halo Rizki! Senang bertemu denganmu. LangChain adalah framework yang sangat menarik untuk mengembangkan aplikasi yang didukung oleh model bahasa besar (LLM). Saya akan senang membantumu belajar tentang LangChain.\n\nUntuk memulai, mari kita pecah apa yang ingin kamu pelajari. Supaya saya bisa memberikan panduan yang paling relevan, bisakah kamu beritahu saya:\n\n1. **Level Pengalamanmu:** Apakah kamu sudah memiliki pengalaman dengan LLM atau Python? Jika ya, seberapa banyak?\n2. **Tujuan Belajarmu:** Apa yang ingin kamu capai dengan mempelajari LangChain? Apakah kamu ingin membangun chatbot, alat analisis data, atau sesuatu yang lain?\n3. **Area Fokus:** Apakah ada area tertentu dari LangChain yang membuatmu tertarik, seperti Rantai (Chains), Agen (Agents), Memori (Memory), atau Integrasi dengan berbagai LLM?\n\nSambil menunggu jawabanmu, berikut adalah beberapa topik dasar yang akan kita bahas:\n\n**Apa itu LangChain?**\n\nLangChain adalah framework yang dirancang u

In [ ]:
# tipe tipe message
# human message
# system message
# ai message

In [9]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates Indonesian into English."),
    HumanMessage(content="Halo nama saya rizki")
]

response = llm.invoke(messages)

In [11]:
response.content

'Hello, my name is Rizki.'

In [13]:
response.usage_metadata

{'input_tokens': 16,
 'output_tokens': 9,
 'total_tokens': 25,
 'input_token_details': {'cache_read': 0}}

In [14]:
# prompt template (ChatPromptTemplate)

from langchain_core.prompts.chat import ChatPromptTemplate

In [15]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Kamu adalah guru multitalenta, berpengalaman di {subjek}"),
    ("human", "Halo jelaskan topik {topik} dengan gaya bahasa {gaya_bahasa}")
])

formatted_prompt = prompt_template.format_messages(
    subjek="matematika",
    topik="logika",
    gaya_bahasa="santai dan mudah dimengerti"
)

formatted_prompt

[SystemMessage(content='Kamu adalah guru multitalenta, berpengalaman di matematika', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Halo jelaskan topik logika dengan gaya bahasa santai dan mudah dimengerti', additional_kwargs={}, response_metadata={})]

In [16]:
response = llm.invoke(formatted_prompt)

response.content

'Oke, siap! Mari kita bahas logika dengan bahasa santai dan mudah dimengerti. Anggap saja kita lagi ngobrol sambil minum kopi, ya. 😉\n\n**Logika itu Apa, Sih?**\n\nGampangnya, logika itu adalah cara berpikir yang masuk akal. Ini tentang bagaimana kita menarik kesimpulan yang benar berdasarkan informasi yang kita punya. Logika membantu kita:\n\n*   **Berpikir Jernih:** Menghindari kebingungan dan kesalahan dalam berpikir.\n*   **Membuat Keputusan yang Tepat:** Mempertimbangkan semua fakta sebelum mengambil tindakan.\n*   **Berargumen dengan Baik:** Menyusun alasan yang kuat untuk mendukung pendapat kita.\n*   **Menganalisis Informasi:** Memilah informasi yang penting dan relevan dari yang tidak.\n\n**Kenapa Logika Itu Penting?**\n\nCoba bayangkan kalau kita nggak pakai logika:\n\n*   Kita gampang percaya berita bohong (hoax).\n*   Kita sering salah paham dengan orang lain.\n*   Kita sulit memecahkan masalah sehari-hari.\n*   Kita mudah dimanipulasi oleh orang lain.\n\nNah, dengan belaja

## Chain (LCEL)

In [24]:
from langchain_core.output_parsers.string import StrOutputParser

In [30]:
lcel_prompt = ChatPromptTemplate.from_messages([
    ("system", "kamu adalah asisten pintar yang menguasai banyak bahasa dan tugas kamu adalah untuk menerjemahkan teks (tanpa meanambahkan apapun)"),
    ("human", "terjemahkan kalimat berikut ke dalam bahasa {bahasa_target}: {teks}")
])

In [31]:
translation_chain = lcel_prompt | llm | StrOutputParser() #LCEL

In [32]:
response = translation_chain.invoke({
    "bahasa_target": "inggris",
    "teks": "Halo nama saya Rizki Teguh"
})

In [33]:
response

'Hello, my name is Rizki Teguh.'

## LCEL yang lebih kompleks

In [34]:
from langchain_core.runnables.passthrough import RunnablePassthrough

In [35]:
# sentiment analysis

sentiment_prompt = ChatPromptTemplate.from_messages([
    ("system", "kamu adalah asisten yang dapat menganalisa sentimen dari sebuah teks. Jawab hanya dengan: 'positif', 'negatif', atau 'netral'"),
    ("human", "analisa sentimen dari teks berikut: {teks}")
])

advice_prompt = ChatPromptTemplate.from_messages([
    ("system", "kamu adalah asisten yang memberikan saran berdasarkan sentimen teks yang diberikan"),
    ("human", "berikan saran berdasarkan sentimen berikut: {sentimen}. \n teks asli: {teks}")
])

In [37]:
sentiment_chain = (
    {"teks": RunnablePassthrough()}
    | sentiment_prompt
    | llm
    | StrOutputParser()
)

In [38]:
advice_chain = (
    {"sentimen": sentiment_chain, "teks": RunnablePassthrough()}
    | advice_prompt
    | llm
    | StrOutputParser()
)

In [39]:
teks_komentar = "saya sangat kecewa dengan pelayanan yang diberikan, sangat buruk dan tidak memuaskan karena lama menunggu dan makanannya sudah dingin"

In [40]:
sentiment = sentiment_chain.invoke({"teks": teks_komentar})

In [41]:
sentiment

'negatif'

In [42]:
advice = advice_chain.invoke({"teks": teks_komentar})

In [43]:
advice

'Karena sentimennya negatif (kekecewaan atas pelayanan yang buruk, waktu tunggu yang lama, dan makanan yang dingin), berikut beberapa saran yang bisa diberikan:\n\n**Saran untuk penyedia layanan (misalnya, restoran):**\n\n*   **Perbaiki Kualitas Pelayanan:**\n    *   Latih staf untuk lebih responsif, ramah, dan efisien.\n    *   Tetapkan standar pelayanan yang jelas dan pastikan semua staf mematuhinya.\n*   **Optimalkan Waktu Tunggu:**\n    *   Evaluasi proses pemesanan dan penyajian makanan untuk mengidentifikasi penyebab keterlambatan.\n    *   Pertimbangkan untuk menambah staf pada jam-jam sibuk.\n    *   Berikan perkiraan waktu tunggu yang akurat kepada pelanggan saat mereka memesan.\n*   **Jaga Kualitas Makanan:**\n    *   Pastikan makanan disajikan dalam keadaan panas dan segar.\n    *   Periksa suhu makanan sebelum disajikan.\n    *   Perbaiki proses persiapan dan penyajian makanan agar tidak terlalu lama menunggu setelah dimasak.\n*   **Tangani Keluhan dengan Profesional:**\n  

## Streaming

In [47]:
prompt = "Bikin cerita satu paragraf tentang robot yang hidup di masa depan"

for chunk in llm.stream(prompt):
    print(chunk.content, end="", flush=True)

Di tengah gemerlap kota futuristik Neo-Tokyo, Unit 734 atau "Rusty" sebagaimana ia dipanggil oleh segelintir teman manusianya, melangkah dengan kaki-kaki mekaniknya yang sedikit berderit di antara kerumunan cyborg dan manusia yang sibuk. Sebagai robot pekerja konstruksi yang sudah usang, Rusty menyaksikan perubahan kota dengan perasaan campur aduk; gedung-gedung pencakar langit yang menjulang semakin tinggi dibangun oleh robot-robot generasi baru yang lebih canggih, sementara ia sendiri semakin terpinggirkan, hanya bisa berharap suatu hari nanti ia akan menemukan tempat di dunia yang terus bergerak maju ini, mungkin sebagai penjaga taman atau bahkan sekadar teman bagi anak-anak manusia yang penasaran dengan sejarah robot-robot tua.


In [48]:
response = llm.invoke('Hello nama saya rizki')
response.content

'Halo Rizki! Senang bertemu denganmu. Ada yang bisa saya bantu?'

In [49]:
response = llm.invoke('Bisa ulangi nama saya siapa?')
response.content

'Sebagai model bahasa AI, saya tidak memiliki ingatan tentang percakapan sebelumnya kecuali Anda menyediakannya dalam percakapan saat ini. Oleh karena itu, saya tidak tahu nama Anda.\n\nAnda bisa memberi tahu nama Anda sekarang jika Anda mau.'

In [52]:
messages = [
    HumanMessage(content="Halo nama saya Rizki"),
    AIMessage(content="Halo Rizki, senang bertemu denganmu!"),
    HumanMessage(content="Bisa ulangi nama saya siapa?")
]

response = llm.invoke(messages)
response.content

'Tentu, nama kamu Rizki.'